In [2]:

from __future__ import print_function, division
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.nn import init
import torch.nn.functional as F
import functools
import numpy as np
import torchvision
from torchvision import transforms, datasets, models
import os
import time

import matplotlib.pyplot as plt

In [40]:
# Training settings
batch_size = 10000

In [41]:
#%%

# p = Augmentor.Pipeline()
# p.random_erasing(0.5,0.4)




train_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomCrop((32, 32), padding=4),   #left, top, right, bottom
#     p.torch_transform(),
#     p.sample(10000),
    transforms.ToTensor()
])
test_transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                               train=True,
                               transform=train_transform,
                               download=True)

test_dataset = datasets.CIFAR10(root='/home/lee/Research/11.project_tangqing/data',
                              train=False,
                              transform=test_transform)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, # 64
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

dataloaders = {
    'train': train_loader,
    'test' : test_loader    
}
image_datasets = {
    'train': train_dataset,
    'test' : test_dataset
}
dataset_sizes = {
    'train': len(image_datasets['train']),
    'test' : len(image_datasets['test'])
}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


print('train_size: ',dataset_sizes['train'],'\ntest_size : ',dataset_sizes['test'])
print(classes)
print('len class: ', len(classes))



Files already downloaded and verified
train_size:  50000 
test_size :  10000
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
len class:  10


In [42]:
# for n, class_n in enumerate(classes):
#     a = 'class_n_{}'.format(class_n)
#     print(n, a)

0 class_n_plane
1 class_n_car
2 class_n_bird
3 class_n_cat
4 class_n_deer
5 class_n_dog
6 class_n_frog
7 class_n_horse
8 class_n_ship
9 class_n_truck


In [100]:
import cv2
import random
# # ADDING NOISE
noise = []
noise = np.random.rand(32, 32,3)
import pickle

# pickle_out = open("batch1","wb")
data = []
filenames = []
occludes = []

for n in range(5):
    inputs, labels = next(iter(dataloaders['train']))
#     print(inputs.shape)
    pickle_out = open("batch{}".format(n+1),"wb")
    data = []
    count = [0 for i in range(10)]
    for m in range(batch_size):
        images_ = inputs[m].numpy()
        class_  = labels[m].numpy()
        images_ = np.transpose(images_, [1,2,0])
        for n, class_n in enumerate(classes):
            if classes[class_] == class_n:
                count[class_] += 1
                if count[class_] < 200:
                    x_random = np.random.randint(6, 27, size=1, dtype='uint8')
                    y_random = np.random.randint(6, 27, size=1, dtype='uint8')
                    w_random = np.random.randint(8, 13, size=1, dtype='uint8')
                    h_random = np.random.randint(8, 13, size=1, dtype='uint8')
                    images_[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]=noise[y_random[0]-int(h_random[0]/2):y_random[0]+int(h_random[0]/2),x_random[0]-int(w_random[0]/2):x_random[0]+int(w_random[0]/2),:]

                    r = images_[:,:,0]
                    g = images_[:,:,1]
                    b = images_[:,:,2]

                    images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
                    images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
                    images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            #             encode_img = images_.tostring()
                    data.append(images_)
                    filename = '{}_{}.png'.format(classes[class_],class_*10000+m)
                    filenames.append(filename)
                    occlude = 1
                    occludes.append(occlude)

#                     plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_],class_*10000+m), images_)

                else:
                    r = images_[:,:,0]
                    g = images_[:,:,1]
                    b = images_[:,:,2]

                    images_[:,:,0] = (r - r.min()) / (r.max() - r.min())
                    images_[:,:,1] = (g - g.min()) / (g.max() - g.min())
                    images_[:,:,2] = (b - b.min()) / (b.max() - b.min())
            #             encode_img = images_.tostring()
                    data.append(images_)
                    filename = '{}_{}.png'.format(classes[class_],class_*10000+m)
                    filenames.append(filename)
#         print(classes[class_.numpy()])
#                     plt.imsave('/home/lee/Research/11.project_tangqing/data/cifar-10-occlude/{}_{}.png'.format(classes[class_],class_*10000+m), images_)

                    occlude = 0
                    occludes.append(occlude)
#         plt.imshow(images_)
#         plt.show()
        
        
    pickle_cifar10 = {'labels': labels.numpy(),'data': data, 'filenames': filenames, 'occludes': occludes}
    pickle.dump(pickle_cifar10, pickle_out)
    pickle_out.close()
    data.clear()
    filenames.clear()
    count.clear()
    occludes.clear()



In [52]:
def unpickle(file):
#     import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [111]:
pickle_in = unpickle("/home/lee/Research/11.project_tangqing/batch1")
# print(len(pickle_in['occludes']))
# print(pickle_in['occludes'])

# print(a[100])
print(pickle_in['data'][0])
print(pickle_in['labels'])
print(pickle_in['filenames'])


# a = pickle_in['occludes']
# count1 = 0
# for i in range(10000):
#     if pickle_in['labels'][i] == 0:
#         if pickle_in['occludes'][i] == 1:
#             count1 += 1
# print(count1)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

